In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.oracle_sql import * 

In [56]:
import re
def define_table(table_name : str ) :
    return f"FROM {table_name}"


def define_select_column(cols : list) :
    if len(cols) == 0 :
        cols = "*"
    else :
        cols = ", ".join(cols)
    return f"SELECT {cols}"

def define_group( cols:list) :
    if len(cols) == 0 :
        return ""
    else :
        cols = ", ".join(cols)
        return f"GROUP BY {cols}"

def define_order( order_cols : list, sorting_types : list = ['DESC']) :
    if len(order_cols) == 0 :
        return ""
    
    assert len(order_cols) == len(sorting_types), "ORDER에 맞게 SORTING TYPE 정해야 함"
    return ", ".join([ f"{col} {type}" for col, type in zip(order_cols , sorting_types)]).strip()


def make_query(select_c, from_c , order_c="" ,  group_c="", where_c="", upper=True) :
    query = f"{select_c} {from_c} {where_c} {group_c} {order_c}"
    if upper :
        query = query.upper()
    return  re.sub(' +', ' ', query).strip()



class WhereQuery(object) : 

    def __init__(self, col , table_name=None) :
        self._col = col
        if table_name is None :
            pass 
        else :
            self._col = f"{table_name}.{self._col}"
    
    def q_between(self,low,high , reverse=False):
        
        between_query = f"{self._col} BETWEEN {low} AND {high}"

        if reverse :
            return "NOT {between_query} "

        else :
            return between_query

    def q_greather(self,low,equal=False) :
        if equal :
            return f"{self._col} >= {low}"
        else :
            return f"{self._col} > {low}"

    def q_lower(self,high , equal=False) :
        if equal :
            return f"{self._col} < {high}"
        else :
            return f"{self._col} <= {high}"

    def q_is_null(self, reverse=False) :
        if reverse :
            return f"{self._col} IS NOT NULL"
        else :
            return f"{self._col} IS NULL"

    def q_equal(self, key, reverse=False) :
        if reverse :
            return f"{self._col} != {key}"
        else :
            return f"{self._col} = {key}"

    def q_in(self, list_value, reverse=False) :
        candidates = ", ".join(list_value)
        if reverse :
            return f"{self._col} NOT IN ({candidates})"
        else :
            return f"{self._col} IN ({candidates})"

    def q_exists(self, sub_query, reverse=False) :
        if reverse :
            return f"NOT EXISTS {sub_query}"
        else :
            return f"EXISTS {sub_query}"

    def q_like(self, pattern) :
        return f"{self._col} LIKE {pattern}"

    @staticmethod
    def add_condition(query1 , query2, method) :
        assert method in ["AND","OR"] , "AND or OR 만 가능합니다."
        return f"{query1} {method} {query2}"

    @staticmethod
    def finish_query(query) :
        return f"WHERE {query}"

def define_temp_table(temp_table_name , query) :
    return f"WITH ({temp_table_name} AS {query})"

def concat_query(query1, query2) :
    q_cat =f"""
    {query1}
    {query2}"""
    return q_cat

def finish_query(query) :
    return f"{query};"


from_clause = define_table(table_name = "A_TABLE")
select_clause = define_select_column(cols = ["A_COLUMN","COUNT(*) AS COUNT_INFO"])
group_clause = define_group(cols=["A_COLUMN"])

where_sql = WhereQuery("a_column")
where_q1 = where_sql.q_between(10,15)
where_q2 = where_sql.q_greather(10)
where_q3 = where_sql.add_condition(where_q1 , where_q2,"AND")
where_clause = where_sql.finish_query(where_q3)

group_count_table = make_query(select_c=select_clause,from_c=from_clause,group_c=group_clause,where_c=where_clause)
temp_table_query = define_temp_table("CT", group_count_table)

from_clause = define_table(table_name = "CT")
select_clause = define_select_column(cols = ["CT.A_COLUMN","CT.COUNT_INFO","SUM(CT.COUNT_INFO) AS TOTAL" , "CT.COUNT_INFO / TOTAL AS RATIO"])
summary_query = make_query(select_c=select_clause,from_c=from_clause)

print(finish_query(concat_query(temp_table_query,summary_query)))



    WITH (CT AS SELECT A_COLUMN, COUNT(*) AS COUNT_INFO FROM A_TABLE WHERE A_COLUMN BETWEEN 10 AND 15 AND A_COLUMN > 10 GROUP BY A_COLUMN)
    SELECT CT.A_COLUMN, CT.COUNT_INFO, SUM(CT.COUNT_INFO) AS TOTAL, CT.COUNT_INFO / TOTAL AS RATIO FROM CT;


'WHERE a_column BETWEEN 10 AND 15 AND a_column > 10'

'a_column > 10'